Sound files taken from 

Sound Effect by <a href="https://pixabay.com/users/dennish18-26151496/?utm_source=link-attribution&utm_medium=referral&utm_campaign=music&utm_content=143040">Dennis</a> from <a href="https://pixabay.com//?utm_source=link-attribution&utm_medium=referral&utm_campaign=music&utm_content=143040">Pixabay</a>

Sound Effect by <a href="https://pixabay.com/users/liecio-3298866/?utm_source=link-attribution&utm_medium=referral&utm_campaign=music&utm_content=190255">LIECIO</a> from <a href="https://pixabay.com//?utm_source=link-attribution&utm_medium=referral&utm_campaign=music&utm_content=190255">Pixabay</a>

Sound Effect from <a href="https://pixabay.com/?utm_source=link-attribution&utm_medium=referral&utm_campaign=music&utm_content=6762">Pixabay</a>

Sound Effect from <a href="https://pixabay.com/sound-effects/?utm_source=link-attribution&utm_medium=referral&utm_campaign=music&utm_content=6064">Pixabay</a>

In [ ]:
!pip install sounddevice simpleaudio pydub pyaudio openai git+https://github.com/openai/whisper.git
!pip install pyaudio openai git+https://github.com/openai/whisper.git 
!pip install python-dotenv soundfile ffmpeg-python scipy

In [3]:
from dotenv import load_dotenv
load_dotenv("D:\\gitFolders\\python_de_learners_data\\.env")

True

In [4]:
from openai import OpenAI
client = OpenAI()

diamond_fallin = "D:\\gitFolders\\python_de_learners_data\\code_script_notebooks\\projects\\audio2text_exploration\\diamond-found-190255.mp3"
crowd_taking = "D:\\gitFolders\\python_de_learners_data\\code_script_notebooks\\projects\\audio2text_exploration\\crowd_talking-6762.mp3"
people_taking = "D:\\gitFolders\\python_de_learners_data\\code_script_notebooks\\projects\\audio2text_exploration\\people-taking-in-small-room.mp3"


In [ ]:
# library playsound did not work 

In [ ]:
# from playsound import playsound
# playsound(crowd_talking)

In [5]:
import simpleaudio as sa

wave_obj = sa.WaveObject.from_wave_file(crowd_taking)  # will error out
play_obj = wave_obj.play()
play_obj.wait_done()  # Wait until sound has finished playing

Error: file does not start with RIFF id

In [6]:
import numpy as np
import simpleaudio as sa

frequency = 440  # Our played note will be 440 Hz
fs = 44100  # 44100 samples per second
seconds = 3  # Note duration of 3 seconds

# Generate array with seconds*sample_rate steps, ranging between 0 and seconds
t = np.linspace(0, seconds, seconds * fs, False)

# Generate a 440 Hz sine wave
note = np.sin(frequency * t * 2 * np.pi)

# Ensure that highest value is in 16-bit range
audio = note * (2**15 - 1) / np.max(np.abs(note))
# Convert to 16-bit data
audio = audio.astype(np.int16)

# Start playback
play_obj = sa.play_buffer(audio, 1, 2, fs)

# Wait for playback to finish before exiting
play_obj.wait_done()

In [7]:
import sounddevice as sd
import soundfile as sf

# Extract data and sampling rate from file
data, fs = sf.read(crowd_taking, dtype='float32')  
sd.play(data, fs)
status = sd.wait()  # Wait until file is done playing

KeyboardInterrupt: 

In [9]:
from pydub import AudioSegment
from pydub.playback import play

sound = AudioSegment.from_mp3(diamond_fallin)
play(sound)

In [10]:
import pyaudio
import wave

# Set chunk size of 1024 samples per data frame
chunk = 1024  

# Open the sound file 
wf = wave.open(diamond_fallin, 'rb')

# Create an interface to PortAudio
p = pyaudio.PyAudio()

# Open a .Stream object to write the WAV file to
# 'output = True' indicates that the sound will be played rather than recorded
stream = p.open(format = p.get_format_from_width(wf.getsampwidth()),
                channels = wf.getnchannels(),
                rate = wf.getframerate(),
                output = True)

# Read data in chunks
data = wf.readframes(chunk)

# Play the sound by writing the audio data to the stream
while data != '':
    stream.write(data)
    data = wf.readframes(chunk)

# Close and terminate the stream
stream.close()
p.terminate()

Error: file does not start with RIFF id

In [12]:
import sounddevice as sd
from scipy.io.wavfile import write

fs = 44100  # Sample rate
seconds = 10  # Duration of recording

myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
sd.wait()  # Wait until recording is finished
write('output.wav', fs, myrecording)  # Save as WAV file

In [17]:
output_wav = "D:\\gitFolders\\python_de_learners_data\\code_script_notebooks\\projects\\audio2text_exploration\\output.wav"

In [18]:
from pydub import AudioSegment
sound = AudioSegment.from_wav(output_wav)

In [19]:
from pydub.playback import play
play(sound)

In [20]:
sound.export('output.mp3', format='mp3')

<_io.BufferedRandom name='output.mp3'>

In [21]:
import pyaudio
import wave

chunk = 1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 2
fs = 44100  # Record at 44100 samples per second
seconds = 3

filename = "pyaudio_output.wav"

p = pyaudio.PyAudio()  # Create an interface to PortAudio

print('Recording')

stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True)

frames = []  # Initialize array to store frames

# Store data in chunks for 3 seconds
for i in range(0, int(fs / chunk * seconds)):
    data = stream.read(chunk)
    frames.append(data)

# Stop and close the stream 
stream.stop_stream()
stream.close()
# Terminate the PortAudio interface
p.terminate()

print('Finished recording')

# Save the recorded data as a WAV file
wf = wave.open(filename, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b''.join(frames))
wf.close()

Recording
Finished recording


In [22]:
pyaudiooutput_wav = "D:\\gitFolders\\python_de_learners_data\\code_script_notebooks\\projects\\audio2text_exploration\\pyaudio_output.wav"

In [23]:
output_audio = open(output_wav, 'rb')

transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=output_audio
)
print(transcription.text)

this is my first recording which is going to be there for quite some time in the output.wav and it's going to be for 10 seconds it's an 8 9


In [1]:
transcription = "this is my first recording which is going to be there for quite some time in the output.wav and it's going to be for 10 seconds it's an 8 9"

import pyttsx3
engine = pyttsx3.init()

engine.say(transcription)
engine.runAndWait()

In [2]:
""" RATE"""
rate = engine.getProperty('rate')   # getting details of current speaking rate
print (rate)                        #printing current voice rate
engine.setProperty('rate', 125)     # setting up new voice rate


"""VOLUME"""
volume = engine.getProperty('volume')   #getting to know current volume level (min=0 and max=1)
print (volume)                          #printing current volume level
engine.setProperty('volume',1.0)    # setting up volume level  between 0 and 1

"""VOICE"""
voices = engine.getProperty('voices')       #getting details of current voice
#engine.setProperty('voice', voices[0].id)  #changing index, changes voices. o for male
engine.setProperty('voice', voices[1].id)   #changing index, changes voices. 1 for female

engine.say(transcription)
engine.say('My current speaking rate is ' + str(rate))
engine.runAndWait()
engine.stop()

"""Saving Voice to a file"""
# On linux make sure that 'espeak' and 'ffmpeg' are installed
engine.save_to_file(transcription, 'test.mp3')
engine.runAndWait()

200
1.0


In [6]:
__file__

NameError: name '__file__' is not defined

In [7]:
from pathlib import Path
from openai import OpenAI
import os
client = OpenAI()

speech_file_path = Path(os.getcwd()).parent / "my_speech.mp3"

response = client.audio.speech.create(
  model="tts-1",
  voice="alloy",
  input=transcription
)

response.stream_to_file(speech_file_path)

C:\Users\kamal\AppData\Local\Temp\ipykernel_260\169414337.py:14: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [8]:
speech_file_path

WindowsPath('d:/gitFolders/python_de_learners_data/code_script_notebooks/projects/my_speech.mp3')